In [28]:
"""
task: Tag value type of variable automatically
author: Zheng Wei
latest modification date: 2018/05/16
"""

"""----------------------PART ONE: Get Return Type-----------------------"""
import numpy as np
import pandas as pd
# load data and grep variables' names.
sample_data = pd.read_csv('C:/Users/zhewei/Desktop/material file/BREAllInOneTrack_10000.csv', sep = '\x07', low_memory=False, encoding='utf-8')
variable_data = pd.read_excel(r'C:/Users/zhewei/Desktop/material file/variables.xlsx')
# sample_variable_name = list(sample_data.iloc[0])
variable_name = list(variable_data['Variable Name'])

In [29]:
import math
import numpy as np
import warnings
warnings.filterwarnings('error')
    
def calc_stat(data):
    """
    Given a column sample data of a variable, calculate its skew and kurt. 
    If the data type is abnormal or sigma of data is zero, we assign 1E7 its skew and kurt.
    
    @para: a column sample data 
    @return: a list, example: [skew, kurt]
    """
    skew = 1E7
    kurt = 1E7
    length = len(data)
    try:
        try:
            miu, miu2 = sum(data)/length, sum(np.array(data)**2)/length
            sigma = math.sqrt(miu2 - miu*miu)
        except:
            pass
        else:
            try:
                miu3 = sum(np.array(data)**3)/length
                if sigma != 0:
                    skew = (miu3 -3*miu*sigma**2-miu**3)/(sigma**3)
            except:
                pass
            finally:
                try:
                    miu4 = sum((np.array(data) - miu)**4)/length
                    if sigma != 0:
                        kurt = miu4/(sigma**4)
                except:
                    pass
        return [skew, kurt]
    except TypeError:
        return [skew, kurt]

In [30]:
def judge(data_name, data):
    """
    Given the sample data of variable, return the list including: 
    return type, missing rate, distinct number and a small sample.
    
    @param data_name: name of a variable
    @param data: sample data of a variable
    
    @return list:[return type, missing rate, distinct numbe, top5 of data]
    """
    
    # part each instance by data form
    string_obj, int_obj, double_obj = [], [], []
    number_nan = 0
    for item in data:
        if item != item:
            number_nan += 1
            continue
        else:
            item = str(item)
        try:
            int_obj.append(int(item))
        except ValueError:
            try:
                double_obj.append(float(item))
            except ValueError:
                string_obj.append(item)
    # check empty data
    if number_nan == len(data):
        return 'NO DATA', 1, 0, 1E7, 1E7, []
 
    len_str, len_int, len_dou = len(string_obj), len(int_obj), len(double_obj)
    
    # compute the values that need to be returned
    missing_rate = number_nan/len(data)
    
    # Avoid 'nan' biases the value of distinct number.
    # distinct_num = len(set(data))
    distinct_num = len(set([item for item in data if item == item])) 
    
    samples = data[:5]
    
    # add skew
    skew = 1E7
    kurt = 1E7
    return_set = [missing_rate, distinct_num, skew, kurt, samples]

    # RULE 1
    if len_str > 0 and str_have_int(string_obj[0]):
        return ['String'] + return_set
    
    if len(set(string_obj))>1 or len_dou + len_int == 0:
        return ['String'] + return_set
    elif len(set(double_obj))>1 or len_int == 0:
        return_set[2:4] = calc_stat(double_obj)
        return ['Double'] + return_set
    else:
        if str_or_int(int_obj):
            return ['String'] + return_set
        else:
            return_set[2:4] = calc_stat(int_obj)
            return ['Integer'] + return_set

In [31]:
def str_have_int(string):
    """
    Judge if a string has integer. Example: "s2" 
    """
    for item in string:
        if item.isdigit():
            return True
    return False

In [32]:
def str_or_int(int_obj):
    """
    Judge if a data with int type starts by 0. Example: '000025'
    """
    for item in int_obj:
        item = str(item)
        if len(item) > 1 and item[0] == '0':
            return True
    return False

In [33]:
predict_return_type = []
missing_rate = []
distinct_num = []
skew = []
kurt = []
samples = []

#  Output the result to excel.
for name in variable_name:
    # index_in_sample_data = sample_variable_name.index(name)
    variable_col_data = list(sample_data[name])  
    return_result = judge(name, variable_col_data) 
    predict_return_type.append(return_result[0])
    missing_rate.append(return_result[1])
    distinct_num.append(return_result[2])
    skew.append(return_result[3])
    kurt.append(return_result[4])
    samples.append(return_result[5])
variable_data['Predict_return_type'] = predict_return_type
variable_data['missing_rate'] = missing_rate
variable_data['distinct_num'] = distinct_num
variable_data['skew'] = skew
variable_data['kurt'] = kurt
variable_data['samples'] = samples
variable_data.to_excel('C:/Users/zhewei/Desktop/predict10000_pri.xlsx')

In [4]:
"""----------------------PART TWO: Get Value Type-----------------------"""

"""
Split word using script provided by Wenkai.
"""

# write variable names into variable_name.txt as input file of split.py.
f = open('C:/Users/zhewei/Desktop/material file/variable_name.txt', 'w')
for name in variable_name:
    f.write(name +'\n')
f.close()

# run split.py and output new_name_tokens_v2.txt
%run split.py

In [18]:
"""
Read the file generated after spilting words and statistical word frenquency.
"""
var_dic = {}  # A dict storing splited word list for each variable. 
word_dic = {}  # key is word and value is its frenqucy. Example: {"ip": 200, "sum": 150...}
f = open('C:/Users/zhewei/Desktop/material file/name_tokens_v2.txt', 'r')
line = f.readline()
while line:
    line = line[:-1].split('\t')
    var_name = line[0]
    word_list = line[1:]
    for item in word_list:
        if len(item) == 1 or item.isdigit():
            continue
        item = item.lower()
        if item in word_dic:
            word_dic[item] += 1
        else:
            word_dic[item] = 1
    var_dic[var_name] = word_list
    line = f.readline()
f.close()

# Sort word_dic by frenqucy.
sort_word_dic = sorted(word_dic.items(), key=lambda item:item[1], reverse=True)

In [19]:
# set the dimension of baseline vector.
vector_dim = 50
base_list = [item[0] for item in sort_word_dic[:vector_dim]]

import numpy as np
# Store vector of variables in var_vector, its key is variable name and value is vector. 
var_vector = {}
for item in var_dic.items():
    temp_vec = np.array([word in item[1] for word in base_list])
    var_vector[item[0]] = list(temp_vec.astype(int))

In [26]:
"""------------------------MACHINE LEARNING------------------------"""

from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import numpy as np

classifiers = [
#         GaussianNB(),
#         KNeighborsClassifier(30),
#         # the number of neighbours
#         # SVC(kernel="linear", C=0.025),
#         SVC(gamma='auto', C=1),
        tree.DecisionTreeClassifier(max_depth=20),
        # RandomForestClassifier(max_depth=20, n_estimators=10, max_features=1),
    ]


In [23]:
def ml(Xtrain, ytrain, Xtest):
    """
    Using different machine learning functions to train labeled dataset
    and then employ the model on all data.
    
    @param Xtrain: Labeled dataset 
    @param ytrain: Manually labeled result.
    @param Xtest: All data
    
    @return result: A dict, its key is model name and value is corresponded result.
    """
    result = {}
    f = open('C:/Users/zhewei/Desktop/ML_result.txt', 'w')
    for model in classifiers:
        time_start = time.time()
        clf = model.fit(Xtrain, ytrain)
        tree.export_graphviz(clf, out_file='C:/Users/zhewei/Desktop/tree.dot')  
        f.write('MODEL: ')
        f.write(str(model))
        f.write('\n')
        # make predictions
        expected = ytrain
        predicted = model.predict(Xtrain)
        result[str(model)] = model.predict(Xtest) 
        # summarize the fit of the model
        f.write('RESULT\n')
        f.write(metrics.classification_report(expected, predicted))
        f.write('Accuracy: ' + str(accuracy_score(expected, predicted)) + '\n')
        print(accuracy_score(expected, predicted))
        f.write('CONFUSION MATRIX\n')
        f.write(str(metrics.confusion_matrix(expected, predicted)) + '\n\n')
        print(str(time.time() - time_start) + ' seconds\n')
    f.close()
    return result

In [24]:
def ml_cor(Xtrain, ytrain, Xtest, ytest, X_end):
    """
    Divide labeled dataset into training dataset and test dataset. 
    Using different machine learning functions to train training dataset, 
    and test the model on test data, eventually employ models on all dataset.   
    
    @param Xtrain: Trainging dataset 
    @param ytrain: Manually labeled result of training dataset.
    @param Xtest: Test dataset
    @param ytest: Manually labeled result of test dataset.
    
    @return result: A dict, its key is model name and value is corresponded result.
    """
    result = {}
    f = open('C:/Users/zhewei/Desktop/ML_result_cor.txt', 'w')
    for model in classifiers:
        time_start = time.time()
        clf = model.fit(Xtrain, ytrain)
        tree.export_graphviz(clf, out_file='C:/Users/zhewei/Desktop/tree_cor.dot')
        f.write('MODEL: ')
        f.write(str(model))
        f.write('\n')
        # make predictions
        expected = ytest
        predicted = model.predict(Xtest)
        result[str(model)] = model.predict(X_end) 
        # summarize the fit of the model
        f.write('RESULT\n')
        f.write(metrics.classification_report(expected, predicted))
        f.write('Accuracy: ' + str(accuracy_score(expected, predicted)) + '\n')
        print(accuracy_score(expected, predicted))
        f.write('CONFUSION MATRIX\n')
        f.write(str(metrics.confusion_matrix(expected, predicted)) + '\n\n')
        print(str(time.time() - time_start) + ' seconds\n')
    f.close()
    return result

In [9]:
"""-----------------------------------divide the dateset for value type-------------------------------------"""
import copy
import pandas as pd
# Update the variable_data using the operated file. 
variable_data2 = pd.read_excel('C:/Users/zhewei/Desktop/predict10000_pri.xlsx')

# Object to be predicted
value_type = list(variable_data2['Value Type'])

# Employed Features
var_name = list(variable_data2['Variable Name'])
return_type = list(variable_data2['Predict_return_type'])
var_type = list(variable_data2['Variable Type'])
distinct_num = list(variable_data2['distinct_num'])
missing_rate = list(variable_data2['missing_rate'])
skew = list(variable_data2['skew'])
kurt = list(variable_data2['kurt'])

# Quantify these two features
var_type_dic = {}
return_type_dic = {}
set_var_type = list(set(var_type))
for i in range(len(set_var_type)):
    var_type_dic[set_var_type[i]] = i

set_return_type = list(set(return_type))
for i in range(len(set_return_type)):
    return_type_dic[set_return_type[i]] = i
    
# Perpare data for function ml()
X_train_data = []
y_train_data = []
X_end = []
for i in range(len(value_type)):
    temp_feature = copy.copy(var_vector[var_name[i]])
    temp_feature += [var_type_dic[var_type[i]], return_type_dic[return_type[i]], distinct_num[i], missing_rate[i], skew[i], kurt[i]]
    if value_type[i] in ('CATEGORICAL', 'CONTINUOUS'):
        X_train_data.append(temp_feature)
        y_train_data.append(value_type[i])
    X_end.append(temp_feature)
assert len(X_train_data) == len(y_train_data)

from sklearn.model_selection import train_test_split
X_train_data_1, X_test_data_1, y_train_data_1, y_test_data_1 = train_test_split(X_train_data, y_train_data, random_state = 4)
# Prepare data for function ml_cor()
# part_point = int(len(X_train_data)*0.8)
# part_point = 7397

In [4]:
"""-----------------------------------Label Tags-------------------------------------------"""
import datetime
import pandas as pd
sample_data = pd.read_csv('C:/Users/zhewei/Desktop/material file/BREAllInOneTrack_10000.csv', sep = '\x07', low_memory=False, encoding='utf-8')
variable_data2 = pd.read_excel('C:/Users/zhewei/Desktop/predict10000_pri.xlsx')
variable_name = list(variable_data2['Variable Name'])

In [13]:
tags = list(variable_data2["Tags"])
for i in range(len(variable_name)):
    variable_col_data = [str(item) for item in list(sample_data[variable_name[i]]) 
                         if (isinstance(item, float) or isinstance(item, int)) and item == item] 
    time_data_num = 0
    no_time_data = []
    for item in variable_col_data:
        if '.' in item:
            temp_item = item[:item.index('.')]
        if len(temp_item) > 10: 
            if len(temp_item) != 13:
                no_time_data.append(item)
                continue
            else:
                item = temp_item[:10]
        try:
            item = float(item)
            readable = datetime.datetime.fromtimestamp(item).isoformat()
            if 1990 <= int(readable[:4]) <= 2050:
                time_data_num += 1
            else:
                no_time_data.append(item)
        except:
            no_time_data.append(item)
    if time_data_num > 0 and len(set(no_time_data)) <= 1:
        tags[i] = "SUP: TIMESTAMP"
variable_data2['Predicted Tags'] = tags
variable_data2.to_excel('C:/Users/zhewei/Desktop/predict10000_tag2.xlsx')

In [83]:
tags = list(variable_data2["Tags"])
used_models = list(variable_data2["Used Models"])
assigned_tags_id = [i for i in range(len(tags)) if (tags[i] == tags[i] or used_models[i] == used_models[i]) 
                    and "MODELSCORE" not in str(tags[i]) and "TIMESTAMP" not in str(tags[i])]

assigned_tags_re = []
temp = []
for id_ in assigned_tags_id:
    if tags[id_] != tags[id_]:
        assigned_tags_re.append("-SUP")
    elif "SUP: NS" in tags[id_]:
        assigned_tags_re.append(tags[id_][tags[id_].index("SUP: "):])
#     elif "SUP: MODELSCORE" in tags[id_] or  "SUP: TIMESTAMP" in tags[id_]: 
#         assigned_tags_id.remove(id_)
    elif "SUP:" in tags[id_]:
        temp.append(id_)
        assigned_tags_re.append("SUP: RAWKEY")
    else:
        assigned_tags_re.append("-SUP")
splited_words = []
for i in range(len(temp)):
    splited_words += [word.lower() for word in var_dic[var_name[temp[i]]] if len(word) > 1 and (not word.isdigit())]

In [115]:
import numpy as np
import pandas as pd
dic_ = dict()
for item in splited_words:
    if item not in dic_:
        dic_[item] = 1
    else:
        dic_[item] += 1
        
vector_dim = 20
base_vec = [item[0] for item in sorted(dic_.items(), key=lambda x: x[1], reverse=True)[:20]]

# Store vector of variables in var_vector, its key is variable name and value is vector. 
var_vector = {}
for item in var_dic.items():
    temp_vec = np.array([word in item[1] for word in base_vec])
    var_vector[item[0]] = list(temp_vec.astype(int))

In [126]:
"""-----------------------------------divide the dateset for model usage-------------------------------------"""
import copy
import pandas as pd
# Update the variable_data using the operated file. 
variable_data2 = pd.read_excel('C:/Users/zhewei/Desktop/predict_all.xlsx')

# Object to be predicted
tags = list(variable_data2["Tags"])
used_models = list(variable_data2["Used Models"])
assigned_tags_id = [i for i in range(len(tags)) if (tags[i] == tags[i] and "MODELSCORE" not in tags[i] 
                    and "TIMESTAMP" not in tags[i]) or used_models[i] == used_models[i]]
0
assigned_tags_re = []
for id_ in assigned_tags_id:
    if tags[id_] != tags[id_]:
        assigned_tags_re.append("-SUP")
    elif "SUP: NS" in tags[id_]:
        assigned_tags_re.append(tags[id_][tags[id_].index("SUP: "):])
#     elif "SUP: MODELSCORE" in tags[id_] or  "SUP: TIMESTAMP" in tags[id_]: 
#         assigned_tags_id.remove(id_)
    elif "SUP:" in tags[id_]:
        assigned_tags_re.append("SUP: RAWKEY")
    else:
        assigned_tags_re.append("-SUP")
        
assert len(assigned_tags_id) == len(assigned_tags_re)
        
# Employed Features
var_name = list(variable_data2['Variable Name'])
return_type = list(variable_data2['Predict_return_type'])
var_type = list(variable_data2['Variable Type'])
distinct_num = list(variable_data2['distinct_num'])
missing_rate = list(variable_data2['missing_rate'])
skew = list(variable_data2['skew'])
kurt = list(variable_data2['kurt'])

# Quantify these two features
var_type_dic = {}
return_type_dic = {}
set_var_type = list(set(var_type))
for i in range(len(set_var_type)):
    var_type_dic[set_var_type[i]] = i

set_return_type = list(set(return_type))
for i in range(len(set_return_type)):
    return_type_dic[set_return_type[i]] = i
    
# Perpare data for function ml()
X_train_data = []
y_train_data = []
X_end = []
for i in range(len(tags)):
    temp_feature = copy.copy(var_vector[var_name[i]])
    temp_feature += [var_type_dic[var_type[i]], return_type_dic[return_type[i]], distinct_num[i], missing_rate[i], skew[i], kurt[i]]
    if i in assigned_tags_id:
        X_train_data.append(temp_feature)
        y_train_data.append(assigned_tags_re[assigned_tags_id.index(i)])
    X_end.append(temp_feature)
assert len(X_train_data) == len(y_train_data)

from sklearn.model_selection import train_test_split
X_train_data_1, X_test_data_1, y_train_data_1, y_test_data_1 = train_test_split(X_train_data, y_train_data, random_state = 4)
# Prepare data for function ml_cor()
# part_point = int(len(X_train_data)*0.8)
# part_point = 7397

In [133]:
# Machine Learning process
import time
result_dic = ml(X_train_data, y_train_data, X_end)
result_dic = ml_cor(X_train_data_1, y_train_data_1, X_test_data_1, y_test_data_1, X_end)

1.0
0.10199642181396484 seconds



In [135]:
# Write the result from Decision Tree into output file.
predict_value_type = []
for (key, value) in result_dic.items():
    if 'DecisionTreeClassifier' in key:
        predict_value_type = value
        break
        
# Write the result from Decision Tree into output file.
output_data = pd.read_excel('C:/Users/zhewei/Desktop/predict_all.xlsx')
predict_value_type = []
for (key, value) in result_dic.items():
    if 'DecisionTreeClassifier' in key:
        predict_value_type = value
        break
output_data['predicted_tags'] = predict_value_type
output_data.to_excel('C:/Users/zhewei/Desktop/end_predict10000v2.1.xlsx')